In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pdfplumber

In [20]:
with pdfplumber.open("Invoice.pdf") as pdf_file:
    data = []    
    for i in range(0, len(pdf_file.pages)):
        page_data = pdf_file.pages[0]
        data.append(page_data.extract_text())

In [23]:
print(data[0].split("\n"))

['8/2/2020 Invoice', 'University of Toronto', '215 Huron Street, 3rd Floor', 'Toronto, Ontario', 'M5S 1A2', 'Tel: (416) 978-2142', 'Fax: (416) 978-2610', 'Email: info.studentaccount@utoronto.ca', 'Account Invoice', 'Student Name: Muhammad Anaqi Muhamad Afendi Last Updated: August 02, 2020', 'Invoice: Fall-Winter 2020-2021', 'Page:     1                                                   20-08-02 22:37:09 ', ' ', ' ', '      ** YOUR CURRENT ACCOUNT IS DISPLAYED HERE IN INVOICE FORMAT. ** ', ' ', 'MR Muhammad Anaqi Muhamad Afendi             ROSI Account : MUHAM1005403332     ', '55 Nassau Street                             Student      : 1005403332          ', 'Toronto ON  M5T 1M3                          Timestamp    : 08/02/2020 22:37:09 ', '=============================================================================== ', 'ROSI Account Balance Up to the End of the Session:                $ 61,719.80   ', '                                                                                

ValueError: seek of closed file